# Study on constraint system implementation

This is my implementation of one-hot encoding section in [Lecture 7 of Armando Solar-Lezama Program Synthesis Course](https://people.csail.mit.edu/asolar/SynthesisCourse/Lecture7.htm) that he uses in Sketch.  

In [1]:
import itertools

# type
Clause = "Literal | Conjunction | Disjunction"

class Literal:
    literal_count = 0
    def __init__(self, truth_value: bool = None, name: str=None):
        self.value = truth_value
        if name:
            self.name = name
        else:
            self.name = f"l{Literal.literal_count}"
            Literal.literal_count += 1

    def update(self, truth_value: bool):
        self.value = truth_value

    def __call__(self):
        return self.value

    def eval(self):
        return self()

    def __str__(self):
        #return f"{self.value}" if self.value is not None else self.name
        return self.name

    def __repr__(self):
        #return f"{self.value}" if self.value is not None else self.name
        return self.name

class Conjunction:
    def __init__(self, left: Clause, right: Clause):
        self.left = left
        self.right = right

    def __call__(self):
        return self.left() and self.right()

    def eval(self):
        return self()

    def __str__(self):
        return f"({self.left} and {self.right})"

    def __repr__(self):
        return f"({self.left} and {self.right})"

class Disjunction:
    def __init__(self, left: Clause, right: Clause):
        self.left = left
        self.right = right

    def __call__(self):
        return self.left() or self.right()

    def eval(self):
        return self()

    def __str__(self):
        return f"({self.left} or {self.right})"

    def __repr__(self):
        return f"({self.left} or {self.right})"

class Value:
    def __init__(self, values_dict: dict[int, Clause]):
        self.values_dict = values_dict

    def __call__(self):
        return {key: val() for key, val in self.values_dict.items()}

    def eval(self):
        return self()

    def __str__(self):
        return f"{self.values_dict}"

    def __repr__(self):
        return f"{self.values_dict}"

    def __add__(self, other):
        possible_values = {}
        for left, right in list(itertools.product(self.values_dict.items(), other.values_dict.items())):
            sum = left[0] + right[0]
            if sum in possible_values.keys():
                previous = possible_values[sum]
                possible_values[sum] = Disjunction(Conjunction(left[1], right[1]), previous)
            else:
                possible_values[sum] = Conjunction(left[1], right[1])

        return Value(possible_values)

In [2]:
# First value composition
b1 = Literal(name="b1")
b2 = Literal(name="b2")
b3 = Literal(name="b3")

val1 = Value({
    5: b1,
    7: b2,
    9: b3
})

# Second value composition
b4 = Literal(name="b4")
b5 = Literal(name="b5")
b6 = Literal(name="b6")

val2 = Value({
    3: b4,
    5: b5,
    7: b6
})

# First demonstration of the classes

We have 3 types of *Clauses*:
- *Literal* - atomic truth variable
- *Conjunction* - $\land$
- *Disjunction* - $\lor$

Formally clause is a collection of literals (or their negations) and logical connectives. To be extra precise, Literal itself is not a Clause, but a set with single Literal is. Here in the implementation it doesn't matter, because from perspective of *Values* both *Conjunctions* and *Disjunctions* evaluate down to the *Literals*.  

*Values* are composed from dictionaries for each possible value.  
In Armando's lecture he refers to them as One-Hot encodings - I'm not sure why, technically they can be represented as one-hot encodings, but those are key-value pairs, so I'm not sure how does one take adventage of keeping keys as one-hot encodings. There's probably reason for that, but I found it more intuitive to implement them as dictionaries (in fact in the lecture notes he represents them as set of tuples as well, which are equivalent to dictionaries).  

Implemented classes work in 2 steps: initialization and evaluation.  

If *Literals* are not initialized with particular truth values, they are represented as None values, but for debugging purposes, *__repr__* and *__str__* show their names instead.  
At this point, if one tries to call/evaluate the Clauses, they'll display None values (which pretty much means they're undetermined).  

Eval/call evaluates the *Value*.  

Only once we pick truth values for *Literals* (i.e. call *literal.update(<truth value>)*), both eval will show the actual values.  
See below...

In [3]:
print("# Before setting truth values\n\n## Truth values\n")
print("".join([f"    {b.name} = {b()}\n" for b in [b1, b2, b3, b4, b5, b6]]))
print(f"## Values - print\n\nPrint shows name of the literal variable for convenience.\n\n    val1={val1}\n    val2={val2}\n")
print(f"## Values - call/eval method\n\n    val1.eval()={val1.eval()}\n    val2.eval()={val2.eval()}")

# Before setting truth values

## Truth values

    b1 = None
    b2 = None
    b3 = None
    b4 = None
    b5 = None
    b6 = None

## Values - print

Print shows name of the literal variable for convenience.

    val1={5: b1, 7: b2, 9: b3}
    val2={3: b4, 5: b5, 7: b6}

## Values - call/eval method

    val1.eval()={5: None, 7: None, 9: None}
    val2.eval()={3: None, 5: None, 7: None}


In [4]:
# let's update some of the literals
b1.update(True)
b3.update(False)
b5.update(False)
b6.update(True)

In [5]:
print("# After updating truth values\n")
print("".join([f"    {b.name} = {b()}\n" for b in [b1, b2, b3, b4, b5, b6]]))
print(f"## Values - print\n\nPrint shows name of the literal variable for convenience.\n\n    val1={val1}\n    val2={val2}\n")
print(f"## Values - call/eval method\n\n    val1.eval()={val1.eval()}\n    val2.eval()={val2.eval()}")

# After updating truth values

    b1 = True
    b2 = None
    b3 = False
    b4 = None
    b5 = False
    b6 = True

## Values - print

Print shows name of the literal variable for convenience.

    val1={5: b1, 7: b2, 9: b3}
    val2={3: b4, 5: b5, 7: b6}

## Values - call/eval method

    val1.eval()={5: True, 7: None, 9: False}
    val2.eval()={3: None, 5: False, 7: True}


In [6]:
# Let's change back before continuing
b1.update(None)
b2.update(None)
b3.update(None)
b4.update(None)
b5.update(None)
b6.update(None)

# Add values together

In [7]:
val3 = val1 + val2

In [8]:
print("# Before updating truth values\n")
print("".join([f"    {b.name} = {b()}\n" for b in [b1, b2, b3, b4, b5, b6]]))
print(f"## Values - print only\n\n    val1 = {val1}\n    val2 = {val2}\n\n    val3 = val1 + val2\n\n    val3 = {val3}\n")

# Before updating truth values

    b1 = None
    b2 = None
    b3 = None
    b4 = None
    b5 = None
    b6 = None

## Values - print only

    val1 = {5: b1, 7: b2, 9: b3}
    val2 = {3: b4, 5: b5, 7: b6}

    val3 = val1 + val2

    val3 = {8: (b1 and b4), 10: ((b2 and b4) or (b1 and b5)), 12: ((b3 and b4) or ((b2 and b5) or (b1 and b6))), 14: ((b3 and b5) or (b2 and b6)), 16: (b3 and b6)}



We can see, upon adding 2 values, we receive constrained results.  
For instance $8$ is a result of adding $5$ and $3$, so if *b1* and *b4* are *True*, that's a valid result.  
Some results can be accomplished in multiple ways - for instance $12$ can be accomplished by adding $5$ and $6$ (if *b1* and *b6* both are *True*) or by adding $7$ and $5$ (if *b2* and *b5* are *True*) or by adding $9$ and $3$ (if *b3* and *b4* are *True*) - they are then connected with disjunction.  

Let's choose some truth values...

In [9]:
b1.update(True)
b2.update(False)
b3.update(True)
b4.update(False)
b5.update(False)
b6.update(True)

In [10]:
print("# After picking truth values\n")
print("".join([f"    {b.name} = {b()}\n" for b in [b1, b2, b3, b4, b5, b6]]))
print(f"## Values - print\n\n    val1={val1}\n    val2={val2}\n    val3={val3}\n")
print(f"## Values - call/eval\n\n    val1.eval()={val1.eval()}\n    val2.eval()={val2.eval()}\n    val3.eval()={val3.eval()}")

# After picking truth values

    b1 = True
    b2 = False
    b3 = True
    b4 = False
    b5 = False
    b6 = True

## Values - print

    val1={5: b1, 7: b2, 9: b3}
    val2={3: b4, 5: b5, 7: b6}
    val3={8: (b1 and b4), 10: ((b2 and b4) or (b1 and b5)), 12: ((b3 and b4) or ((b2 and b5) or (b1 and b6))), 14: ((b3 and b5) or (b2 and b6)), 16: (b3 and b6)}

## Values - call/eval

    val1.eval()={5: True, 7: False, 9: True}
    val2.eval()={3: False, 5: False, 7: True}
    val3.eval()={8: False, 10: False, 12: True, 14: False, 16: True}


And here we can see that under those specific conditions/constraints, the result of addition holds for values:
- $12$ (because *b1* and *b6* are *True*).  
- $16$ (because *b3* and *b6* are *True*).  

Now, just to show why this can be technically represented as one-hot encoding, the same dictionary can be represented like so:

In [11]:
max_num = 20 # represents space of possible results

print([1 if key in val1.values_dict.keys() else 0 for key in range(max_num)])
print([1 if key in val2.values_dict.keys() else 0 for key in range(max_num)])
print([1 if key in val3.values_dict.keys() else 0 for key in range(max_num)])

[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0]


$1$/$0$ for possible number/solution. With corresponding clauses ($0$s below for clarity, those are in fact *False*):

In [12]:
print([val1.values_dict[key] if key in val1.values_dict.keys() else 0 for key in range(max_num)])
print([val2.values_dict[key] if key in val2.values_dict.keys() else 0 for key in range(max_num)])
print([val3.values_dict[key] if key in val3.values_dict.keys() else 0 for key in range(max_num)])

[0, 0, 0, 0, 0, b1, 0, b2, 0, b3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, b4, 0, b5, 0, b6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, (b1 and b4), 0, ((b2 and b4) or (b1 and b5)), 0, ((b3 and b4) or ((b2 and b5) or (b1 and b6))), 0, ((b3 and b5) or (b2 and b6)), 0, (b3 and b6), 0, 0, 0]


And, so each value has it's own corresponding clause - but that's equivalent to dictionary in the implementation.  